In [2]:
import pandas as pd
import re
from rapidfuzz import process, fuzz

In [3]:
bgg=pd.read_csv('archive/games.csv')
kickstarter=pd.read_csv('kickstarter_succesful.csv')

In [4]:
bgg.head()

,BGGId,Name,Description,YearPublished,GameWeight,AvgRating,BayesAvgRating,StdDev,MinPlayers,MaxPlayers,...,Rank:partygames,Rank:childrensgames,Cat:Thematic,Cat:Strategy,Cat:War,Cat:Family,Cat:CGS,Cat:Abstract,Cat:Party,Cat:Childrens
0,1,Die Macher,die macher game seven sequential political rac...,1986,4.3206,7.61428,7.10363,1.57979,3,5,...,21926,21926,0,1,0,0,0,0,0,0
1,2,Dragonmaster,dragonmaster tricktaking card game base old ga...,1981,1.9630,6.64537,5.78447,1.45440,3,4,...,21926,21926,0,1,0,0,0,0,0,0
2,3,Samurai,samurai set medieval japan player compete gain...,1998,2.4859,7.45601,7.23994,1.18227,2,4,...,21926,21926,0,1,0,0,0,0,0,0
3,4,Tal der Könige,triangular box luxurious large block tal der k...,1992,2.6667,6.60006,5.67954,1.23129,2,4,...,21926,21926,0,0,0,0,0,0,0,0
4,5,Acquire,acquire player strategically invest business t...,1964,2.5031,7.33861,7.14189,1.33583,2,6,...,21926,21926,0,1,0,0,0,0,0,0


In [5]:
kickstarter.head()

,Unnamed: 0,id,name,backers_count,converted_pledged_amount,country,deadline,link,percent_funded
0,0,1248099364,ACE COMBAT CATS,17,175,MX,1744657200,https://www.kickstarter.com/projects/141774304...,1972.222222
1,1,929957215,TACTICAL CHIBAS,21,228,MX,1744653600,https://www.kickstarter.com/projects/991612028...,2109.090909
2,2,682721647,LeTable Digital One Touch Board Game Table,1,19,GB,1744296301,https://www.kickstarter.com/projects/letable/l...,150.000000
3,3,1871216649,KNIGHTS OF THE BLACK POND,28,504,MX,1744412400,https://www.kickstarter.com/projects/121428851...,4320.833333
4,4,1759337203,Bowsette Bondage & Branwen 3d Print Collectable,16,260,MX,1744318986,https://www.kickstarter.com/projects/balam3d/b...,5280.000000


In [6]:
bgg.count()

BGGId                  21925
Name                   21925
Description            21924
YearPublished          21925
GameWeight             21925
AvgRating              21925
BayesAvgRating         21925
StdDev                 21925
MinPlayers             21925
MaxPlayers             21925
ComAgeRec              16395
LanguageEase           16034
BestPlayers            21925
GoodPlayers            21925
NumOwned               21925
NumWant                21925
NumWish                21925
NumWeightVotes         21925
MfgPlaytime            21925
ComMinPlaytime         21925
ComMaxPlaytime         21925
MfgAgeRec              21925
NumUserRatings         21925
NumComments            21925
NumAlternates          21925
NumExpansions          21925
NumImplementations     21925
IsReimplementation     21925
Family                  6663
Kickstarted            21925
ImagePath              21908
Rank:boardgame         21925
Rank:strategygames     21925
Rank:abstracts         21925
Rank:familygam

In [7]:
kickstarter.count()

Unnamed: 0                  2239
id                          2239
name                        2239
backers_count               2239
converted_pledged_amount    2239
country                     2239
deadline                    2239
link                        2239
percent_funded              2239
dtype: int64

In [25]:
# Normalize function
def normalize(name):
    name = name.lower()
    name = re.sub(r'[^\w\s]', '', name)  # Remove punctuation
    noise_words = r'\b(deluxe|edition|kickstarter|core|board game|game|box|the|a|and|second|1st|2nd|3rd)\b'
    name = re.sub(noise_words, '', name)
    name = re.sub(r'\s+', ' ', name).strip()  # Remove extra spaces
    return name

In [26]:
bgg['norm_bgg'] = bgg['Name'].apply(normalize)
kickstarter['norm_ks'] = kickstarter['name'].apply(normalize)

In [27]:
ks_names = kickstarter['norm_ks'].tolist()

In [33]:
from rapidfuzz import fuzz, process

matches = []

# For faster lookups
ks_name_map = dict(zip(kickstarter['norm_ks'], kickstarter['name']))

for bgg_row in bgg.itertuples():
    bgg_name = bgg_row.norm_bgg
    original_bgg_name = bgg_row.Name
    matched_name = None
    similarity = 0

    # A. Exact match
    if bgg_name in ks_name_map:
        matched_name = ks_name_map[bgg_name]
        similarity = 100

    else:
        # B. One name contains the other
        for ks_name in ks_name_map:
            if bgg_name in ks_name or ks_name in bgg_name:
                matched_name = ks_name_map[ks_name]
                similarity = 95  # Assign high similarity due to containment
                break

        # C. Token-based fuzzy match (only if no match yet)
        if not matched_name:
            best_match = process.extractOne(bgg_name, ks_name_map.keys(), scorer=fuzz.token_sort_ratio)
            if best_match and best_match[1] >= 75:
                matched_name = ks_name_map[best_match[0]]
                similarity = best_match[1]

    matches.append((original_bgg_name, matched_name, similarity))

# Convert to DataFrame
results = pd.DataFrame(matches, columns=['bgg_name', 'matched_ks_name', 'similarity'])

# Optional: Save
results.to_csv("matched_games.csv", index=False)

In [42]:
results[results['similarity']>=95].sort_values('similarity',ascending=True)

,bgg_name,matched_ks_name,similarity
21842,HIT !,"Aristocratic architecture, STL, 28MM",95.0
338,Ocean,Darwin's Journey: Oceania,95.0
339,Brawl,ANIMALCATRAZ - The Sandbox Prison Brawler,95.0
350,Meteo,The Whitefrost Plague: A Meteor Tales RPG Camp...,95.0
351,Zoon,Zoontalis: Dark Kingdom – Awakened Beasts Batt...,95.0
...,...,...,...
14913,Hoard,HOARD,100.0
11917,Mafia City,Mafia City,100.0
18344,Cosmos,COSMOS,100.0
70,Falling,FALLING,100.0


In [48]:
bgg.size/len(bgg.columns)

21925.0

In [49]:
kickstarter.size/len(kickstarter.columns)

2239.0